In [108]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.models import save_model


In [109]:
def load_dataset(file = 'traindata.txt'):
    with open('/home/farzad/Documents/kaggle/'+file, 'r') as f:
        labels = []
        text = []
        
        lines = f.readlines()
    shuffle(lines)
    for line in lines:
        data = line.split('\t')
        if len(data) == 2:
            labels.append(int(data[0]))
            text.append(data[1].rstrip())
    return text,labels
    
x_train_text , y_train = load_dataset()

In [110]:
data_text = x_train_text
idx = 5
print(x_train_text[idx],'\n',y_train[idx])

I am going to start reading the Harry Potter series again because that is one awesome story. 
 1


In [111]:
import re
def process(txt):
    out = re.sub(r'[^a-zA-Z0-9\s]', '', txt)
    out = out.split()
    out = [word.lower() for word in out]
    return out

In [112]:
def tokenize(thresh = 5):
    count  = dict()
    idx = 1
    word_index = dict()
    for txt in data_text:
        words = process(txt)
        for word in words:
            if word in count.keys():
                count[word] += 1
            else:
                count[word]  = 1
    most_counts = [word for word in count.keys() if count[word]>=thresh]
    for word in most_counts:
        word_index[word] = idx
        idx+=1
    return word_index

In [113]:
num_words = None
word_index = tokenize()
num_words = len(word_index)
print('length of the dictionary ',len(word_index))

length of the dictionary  452


In [114]:
def getMax(data):
    max_tokens = 0 
    for txt in data:
        if max_tokens < len(txt.split()):
            max_tokens = len(txt.split())
    return max_tokens
max_tokens = getMax(x_train_text)

In [115]:
def create_sequences(data):
    tokens = []
    for txt in data:
        words = process(txt)
        seq = [0] * max_tokens
        i = 0 
        for word in words:
            start = max_tokens-len(words)
            if word.lower() in word_index.keys():
                seq[i+start] = word_index[word]
            i+=1
        tokens.append(seq)        
    return np.array(tokens)

In [116]:
print(create_sequences(['awesome movie']))

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0 32 86]]


In [117]:
x_train_tokens = create_sequences(x_train_text)
x_train_tokens[0]


array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 279, 304,  11,
       188])

In [89]:
# model = Sequential()
# embedding_size = 8
# model.add(Embedding(input_dim=num_words,
#                     output_dim=embedding_size,
#                     input_length=max_tokens,
#                     name='layer_embedding'))

# model.add(GRU(units=16, name = "gru_1",return_sequences=True))
# model.add(GRU(units=8, name = "gru_2" ,return_sequences=True))
# model.add(GRU(units=4, name= "gru_3"))
# model.add(Dense(1, activation='sigmoid',name="dense_1"))
# optimizer = Adam(lr=1e-3)
# model.compile(loss='binary_crossentropy',
#               optimizer=optimizer,
#               metrics=['accuracy'])


In [118]:
x_train_tokens[7085].shape
# type(x_train_tokens)
# np.array(y_train)

(40,)

In [121]:
from keras import layers
from keras.models import Sequential
from keras.utils import to_categorical
from keras import models


model = Sequential() # Creating Model By Adding Layers One By One

# Input - Layer
model.add(layers.Dense(40, activation = "relu", input_shape=(40, )))

# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(40, activation = "relu"))
          
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(40, activation = "relu"))

# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))

model.summary()



model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 40)                1640      
_________________________________________________________________
dropout_1 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 40)                1640      
_________________________________________________________________
dropout_2 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 40)                1640      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 41        
Total params: 4,961
Trainable params: 4,961
Non-trainable params: 0
_________________________________________________________________


In [126]:
results = model.fit(
 x_train_tokens, np.array(y_train),
 epochs= 10,
 batch_size = 16,
 validation_split=.1
)
print("Test-Accuracy:", np.mean(results.history["val_acc"]))

Train on 6377 samples, validate on 709 samples
Epoch 1/10
6377/6377 [==============================] - 1s 106us/step - loss: 0.3039 - acc: 0.8827 - val_loss: 0.1898 - val_acc: 0.9351
Epoch 2/10
6377/6377 [==============================] - 1s 105us/step - loss: 0.3091 - acc: 0.8822 - val_loss: 0.1943 - val_acc: 0.9351
Epoch 3/10
6377/6377 [==============================] - 1s 106us/step - loss: 0.2907 - acc: 0.8890 - val_loss: 0.1981 - val_acc: 0.9365
Epoch 4/10
6377/6377 [==============================] - 1s 131us/step - loss: 0.2994 - acc: 0.8929 - val_loss: 0.2104 - val_acc: 0.9267
Epoch 5/10
6377/6377 [==============================] - 1s 105us/step - loss: 0.2713 - acc: 0.8982 - val_loss: 0.1893 - val_acc: 0.9323
Epoch 6/10
6377/6377 [==============================] - 1s 103us/step - loss: 0.2693 - acc: 0.9017 - val_loss: 0.1913 - val_acc: 0.9323
Epoch 7/10
6377/6377 [==============================] - 1s 101us/step - loss: 0.2643 - acc: 0.9034 - val_loss: 0.1816 - val_acc: 0.9337
E

In [127]:
res2 = model.fit(np.array(x_train_tokens),np.array(y_train),validation_split=0.05, epochs=5, batch_size=32)
print("Test-Accuracy:", np.mean(res2.history["val_acc"]))

Train on 6731 samples, validate on 355 samples
Epoch 1/5
6731/6731 [==============================] - 0s 63us/step - loss: 0.2271 - acc: 0.9171 - val_loss: 0.1804 - val_acc: 0.9380
Epoch 2/5
6731/6731 [==============================] - 0s 56us/step - loss: 0.2365 - acc: 0.9135 - val_loss: 0.1716 - val_acc: 0.9352
Epoch 3/5
6731/6731 [==============================] - 0s 57us/step - loss: 0.2339 - acc: 0.9120 - val_loss: 0.1615 - val_acc: 0.9465
Epoch 4/5
6731/6731 [==============================] - 0s 55us/step - loss: 0.2296 - acc: 0.9132 - val_loss: 0.1685 - val_acc: 0.9268
Epoch 5/5
6731/6731 [==============================] - 0s 69us/step - loss: 0.2241 - acc: 0.9137 - val_loss: 0.1678 - val_acc: 0.9324
Test-Accuracy: 0.9357746478873239


In [132]:
x_test, y_test = load_dataset()
x_test_tokens = create_sequences(x_test)

#x_test_tokens

score, acc = model.evaluate(x_test_tokens, y_test,batch_size=16)
print('Test score:', score)
print('Test accuracy:', acc)

7086/7086 [==============================] - 0s 38us/step
Test score: 0.13834074508023403
Test accuracy: 0.9458086366980486
